In [1]:
import pandas as pd
import numpy as np
import itertools
import os
from pulp import *

In [2]:
problem = LpProblem("E-Scooter Allocation", LpMinimize)

In [3]:
df_distance = pd.read_csv("Data/Model Data - Distance Matrix.csv")
df_distance.fillna(0, inplace=True)
distance = df_distance.values[:,1:]
distance_dict = { (x,y): distance[x][y] for x in range(distance.shape[0]) for y in range(distance.shape[1])}

In [4]:
locations = list(df_distance['Location'])
location_idx = np.arange(0, len(locations))
loc_count = len(locations)

In [5]:
demand = dict()
files = os.listdir('Data/Demand/')
for i, f in enumerate(files):
    hour = int(f.split(sep='.', maxsplit=1)[0])
    df = pd.read_csv('Data/Demand/' + f)
    demand_arr = df.values[:-1,1:-1]
    demand[hour] = demand_arr

In [6]:
demand_dict = dict()
for i,(_,v) in enumerate(demand.items()):
    demand_dict.update({(x,y,i): v[x][y] for x in range(loc_count) for y in range(loc_count)})

In [7]:
# Parameters
M = sys.maxsize
Nmax = 40
Vmax = 20
# Cost values
C_scooter_km = 42
C_mf_scooter = 16
C_mf_dock = 30
# Cost relocation
C_r = 160

In [8]:
# Penalty
df_penalty = pd.read_csv("Data/Penalty Carbon Costs.csv")
C_pen = df_penalty.values[:-1, 1]

In [9]:
# Sets
T = np.arange(0,len(files))
# each location at a given time
X = list(itertools.product(location_idx, T))
A1 = [(xi, xj) for xi in X for xj in X if xi[0] != xj[0] and xi[1]+1==xj[1]]
A2 = [(xi, xj) for xi in X for xj in X if xi[0]==xj[0] and xi[1]+1==xj[1]]
# Relocation
A3 = [(xi, xj) for xi in X for xj in X if xi[0]!=xj[0] and xi[1]==T[-1] and xj[1]==T[0]]

In [10]:
T

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [11]:
# Decision Variables
Yi = LpVariable.dicts("Station Presence", location_idx,0,cat=const.LpBinary)
Zi = LpVariable.dicts("Size", location_idx, 0, cat="Continuous")
# Relocation
Rij = LpVariable.dicts("#Scooters", A3, 0, cat=const.LpInteger)
Vit = LpVariable.dicts("#Available_scooters",X,0,cat="Continuous")
Sit = LpVariable.dicts("#Stocked_Scooters", A2, 0, cat="Continuous")
Ditj = LpVariable.dicts("#Used_Scooters", A1, 0 ,cat="Continuous")

In [12]:
# Objective function
problem+=lpSum(Ditj[((i,ti),(j,tj))]*distance_dict[(i,j)]*C_scooter_km for ((i,ti),(j,tj)) in A1) + \
    lpSum((demand_dict[(i,j,ti)]-Ditj[((i,ti),(j,tj))])*distance_dict[(i,j)]*C_pen[i] for ((i,ti),(j,tj)) in A1) + \
    lpSum(Rij[((i,ti), (j,tj))]*distance_dict[(i,j)]*C_r for ((i,ti), (j,tj)) in A3) + \
    lpSum(Vit[(i,t)]*C_mf_scooter for (i,t) in X if t==0) + \
    lpSum(Yi[i]*C_mf_dock for i in location_idx), "Objective function"

In [13]:
for (i,ti) in X:
    if ti !=0:
        problem+= Vit[(i,ti-1)] - lpSum(Ditj[((i,ti-1),(j,tj))] for (j,tj) in X if tj==ti and i !=j) + lpSum(Ditj[((j, tj),(i, ti))] for (j,tj) in X if tj == ti-1 and i !=j) == Vit[(i,ti)], f"Availability Balance {(i,ti)}"


In [14]:
for (i,ti) in X:
    if ti!=T[-1]:
        problem+=Vit[(i,ti)] - lpSum(Ditj[((i,ti),(j,tj))]for (j,tj) in X if i !=j and ti+1==tj) == Sit[((i,ti), (i,ti+1))], f"Stocked Scooters for {(i,ti)}"

In [15]:
# Simplified relocation constraint
for i in location_idx:
    problem+=Vit[(i,0)] == Vit[(i,T[-1])] + lpSum(Rij[((j,T[-1]), (i,0))] for j in location_idx if i!=j) - lpSum(Rij[((i,T[-1]), (j,0))] for j in location_idx if i!=j), f"Relocation balance for location {i}"

In [16]:
for i in location_idx:
    problem+=lpSum(Rij[((i,T[-1]), (j,0))] for j in location_idx if i!=j) <= Vit[(i,T[-1])], f"Relocation availability for location {i}"

In [17]:
for (i,ti) in X:
    problem+= Zi[i] >=Vit[(i,ti)], f"Size constraint for {(i,ti)}"

In [18]:
for (i,ti) in X:
    problem+=Vit[(i,ti)] <= M*Yi[i], f"Availability against Dock present{(i,ti)}"

In [19]:
for i in location_idx:
    problem+=Yi[i] <= Zi[i], f"Size constraint for {i}"

In [20]:
for ((i,ti),(j,tj)) in A1:
    problem+=Ditj[((i,ti),(j,tj))]<=demand_dict[(i,j,ti)], f"Maximum Demand for {((i,ti),(j,tj))}"

In [21]:
for i in location_idx:
    problem+=Zi[i] <= Smax, f"Maximum size for {i}"

In [22]:
problem+=lpSum(Yi[i] for i in location_idx) <= Nmax, f"Maximum number of docks"

In [23]:
problem.writeLP("E-scooterProblem.lp")
problem.solve()
LpStatus[problem.status]

'Optimal'

In [27]:
for v in problem.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

8,_7),_(80,_8)) = 1.0
#Used_Scooters_((98,_7),_(81,_8)) = 1e-10
#Used_Scooters_((98,_7),_(82,_8)) = 1.0
#Used_Scooters_((98,_7),_(83,_8)) = 1e-10
#Used_Scooters_((98,_7),_(86,_8)) = 1e-10
#Used_Scooters_((98,_7),_(87,_8)) = 1e-10
#Used_Scooters_((98,_7),_(88,_8)) = 1e-10
#Used_Scooters_((98,_7),_(9,_8)) = 1e-10
#Used_Scooters_((98,_7),_(92,_8)) = 1.0
#Used_Scooters_((98,_7),_(94,_8)) = 1e-10
#Used_Scooters_((98,_7),_(95,_8)) = 2.0100009e-10
#Used_Scooters_((98,_8),_(0,_9)) = 1e-10
#Used_Scooters_((98,_8),_(100,_9)) = 9.9e-11
#Used_Scooters_((98,_8),_(101,_9)) = 2e-10
#Used_Scooters_((98,_8),_(103,_9)) = 1e-10
#Used_Scooters_((98,_8),_(104,_9)) = 1.0
#Used_Scooters_((98,_8),_(107,_9)) = 9.9e-11
#Used_Scooters_((98,_8),_(11,_9)) = 9.9e-11
#Used_Scooters_((98,_8),_(110,_9)) = 1e-10
#Used_Scooters_((98,_8),_(12,_9)) = 1e-10
#Used_Scooters_((98,_8),_(13,_9)) = 1e-10
#Used_Scooters_((98,_8),_(14,_9)) = 1e-10
#Used_Scooters_((98,_8),_(15,_9)) = 1e-10
#Used_Scooters_((98,_8),_(16,_9)) = 1e-10


In [29]:
value(problem.objective)

5650676.64751113

In [30]:
# f = open("obj.txt", "w+")
# print(problem.objective, file=f)
# f.close()

In [31]:
# Extracting the results
#(idx, LpVariable)
size_list = []
for _,v in Zi.items():
    size_list.append(v.value())  

In [32]:
station_list = []
for _,v in Yi.items():
    station_list.append(v.value()) 

In [33]:
df_results = pd.DataFrame(list(zip(locations, station_list, size_list)), columns=["Location","Station", "Size"])
df_results.head()

,Location,Station,Size
0,Abbeyhill,0.0,2.105355
1,Baberton and Juniper Green,1.0,20.000000
2,Balerno and Bonnington Village,1.0,20.000000
3,Balgreen and Roseburn,0.0,9.999920
4,"Barnton, Cammo and Cramond South",0.0,20.000085


In [34]:
df_results.to_csv("results2.csv", index=False)